<a href="https://colab.research.google.com/github/hayata1996/kaggle/blob/main/KaggleObesity240309.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:00.09685Z","iopub.execute_input":"2024-02-23T07:46:00.097531Z","iopub.status.idle":"2024-02-23T07:46:12.631395Z","shell.execute_reply.started":"2024-02-23T07:46:00.097495Z","shell.execute_reply":"2024-02-23T07:46:12.630278Z"}}
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, roc_curve, accuracy_score, f1_score, auc,classification_report
from scipy.stats import ks_2samp

from sklearn.preprocessing import label_binarize,OneHotEncoder, StandardScaler, FunctionTransformer, LabelEncoder
from itertools import cycle

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import RandomizedSearchCV
import shap

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, PCA

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:12.633174Z","iopub.execute_input":"2024-02-23T07:46:12.635012Z","iopub.status.idle":"2024-02-23T07:46:12.88707Z","shell.execute_reply.started":"2024-02-23T07:46:12.634965Z","shell.execute_reply":"2024-02-23T07:46:12.885267Z"}}
train = pd.read_csv("/kaggle/input/playground-series-s4e2/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s4e2/test.csv")

# %% [code] {"_kg_hide-input":true,"execution":{"iopub.status.busy":"2024-02-23T07:46:12.889916Z","iopub.execute_input":"2024-02-23T07:46:12.890352Z","iopub.status.idle":"2024-02-23T07:46:12.995003Z","shell.execute_reply.started":"2024-02-23T07:46:12.890322Z","shell.execute_reply":"2024-02-23T07:46:12.99338Z"}}
original_data = pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')
train = pd.concat([train, original_data]).drop(['id'], axis=1).drop_duplicates()

# %% [markdown]
# **Creating Column List**

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:12.997175Z","iopub.execute_input":"2024-02-23T07:46:12.997629Z","iopub.status.idle":"2024-02-23T07:46:13.011641Z","shell.execute_reply.started":"2024-02-23T07:46:12.997594Z","shell.execute_reply":"2024-02-23T07:46:13.009577Z"}}
target = 'NObeyesdad'
num_col = []
cat_col = []

for i in train.columns.drop([target]) :

    if train[i].dtype == 'object' :
        cat_col.append(i)

    else :
        num_col.append(i)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:13.013309Z","iopub.execute_input":"2024-02-23T07:46:13.014Z","iopub.status.idle":"2024-02-23T07:46:13.038262Z","shell.execute_reply.started":"2024-02-23T07:46:13.013939Z","shell.execute_reply":"2024-02-23T07:46:13.036518Z"}}
print("Numerical Columns : ", *num_col,"\n",sep="\n")
print("Categorical Columns : ", *cat_col,sep="\n")

# %% [markdown]
# <a id='EDA'></a>
#
# # EDA

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:13.04018Z","iopub.execute_input":"2024-02-23T07:46:13.040633Z","iopub.status.idle":"2024-02-23T07:46:13.0524Z","shell.execute_reply.started":"2024-02-23T07:46:13.0406Z","shell.execute_reply":"2024-02-23T07:46:13.051095Z"}}
basic_EDA = True

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:13.054079Z","iopub.execute_input":"2024-02-23T07:46:13.054511Z","iopub.status.idle":"2024-02-23T07:46:13.251303Z","shell.execute_reply.started":"2024-02-23T07:46:13.054475Z","shell.execute_reply":"2024-02-23T07:46:13.249709Z"}}
if basic_EDA :
    print("\n","="*25,"Train","="*25)
    display(train.describe().T)
    display(train.info())

    print("\n","="*25,"Test","="*25)
    display(test.describe().T)
    display(test.info())

# %% [markdown]
# <a id='EDA2'></a>
#
# # EDA - Multivariate

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:13.253148Z","iopub.execute_input":"2024-02-23T07:46:13.253552Z","iopub.status.idle":"2024-02-23T07:46:13.263492Z","shell.execute_reply.started":"2024-02-23T07:46:13.25352Z","shell.execute_reply":"2024-02-23T07:46:13.261804Z"}}
def corr_heat_map(df,scale=1) :
    # Calculate the correlation matrix
    correlation_matrix = df.corr()

    # Create a mask for the upper triangle
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

    # Set up the matplotlib figure
    plt.figure(figsize=(10//scale, 8//scale))

    # Define a custom color palette
    cmap = sns.diverging_palette(220, 20, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(correlation_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": 0.7})

    plt.title('Correlation Heatmap')

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:13.269957Z","iopub.execute_input":"2024-02-23T07:46:13.27042Z","iopub.status.idle":"2024-02-23T07:46:14.217873Z","shell.execute_reply.started":"2024-02-23T07:46:13.270383Z","shell.execute_reply":"2024-02-23T07:46:14.216423Z"}}
plt.figure(figsize=(20,7))

corr_heat_map(train[num_col],2)
corr_heat_map(test[num_col],2)

plt.show()

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:46:14.219796Z","iopub.execute_input":"2024-02-23T07:46:14.220247Z","iopub.status.idle":"2024-02-23T07:48:24.848823Z","shell.execute_reply.started":"2024-02-23T07:46:14.22021Z","shell.execute_reply":"2024-02-23T07:48:24.847368Z"}}
plt.figure(figsize=(30,30))
temp=num_col.copy()
temp.extend([target])
sns.pairplot(train[sorted(temp)],hue=target)
plt.show()

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:24.850495Z","iopub.execute_input":"2024-02-23T07:48:24.851114Z","iopub.status.idle":"2024-02-23T07:48:29.538361Z","shell.execute_reply.started":"2024-02-23T07:48:24.851071Z","shell.execute_reply":"2024-02-23T07:48:29.536654Z"}}
plt.figure(figsize=(50,4))

x=1
for i in num_col :

    plt.subplot(1,8,x)
    sns.violinplot(
        data=train,
        x=target,
        y=i
    )
    plt.xticks(rotation=45)
    x+=1

plt.show()

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:29.54001Z","iopub.execute_input":"2024-02-23T07:48:29.540393Z","iopub.status.idle":"2024-02-23T07:48:52.587621Z","shell.execute_reply.started":"2024-02-23T07:48:29.540361Z","shell.execute_reply":"2024-02-23T07:48:52.586109Z"}}
temp = train.copy()
plt.figure(figsize=(30,12))
plot_num = 1
for i in ['Age', 'Height', 'FCVC', 'NCP', 'FAF', 'TUE'] :

    sc = StandardScaler()
    p =  PCA(n_components=2,random_state=42)
    temp['Weight2'] = temp['Weight']//5

    x=['Weight2','CH2O',i]
    temp[['pca_1_wt_ch20_'+i+"_1",'pca_1_wt_ch20_'+i+"_2"]] = p.fit_transform(temp[x])

    plt.subplot(2,3,plot_num)
    plt.title(x)
    sns.scatterplot(x= temp['pca_1_wt_ch20_'+i+"_1"],
                    y=temp['pca_1_wt_ch20_'+i+"_2"],
                    hue=temp[target],
                    palette = 'pastel',
                    legend = 'full'
    )
    sns.regplot(x= temp['pca_1_wt_ch20_'+i+"_1"],
                y=temp['pca_1_wt_ch20_'+i+"_2"],
                scatter=False
    )

    plot_num+=1

plt.show()

# %% [markdown]
# <a id='FE'></a>
# # Feature Engineering

# %% [markdown]
# <a id='Label Encoding'></a>
# ### PipeLine

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:52.589386Z","iopub.execute_input":"2024-02-23T07:48:52.589792Z","iopub.status.idle":"2024-02-23T07:48:52.686188Z","shell.execute_reply.started":"2024-02-23T07:48:52.58976Z","shell.execute_reply":"2024-02-23T07:48:52.684657Z"}}
train = pd.get_dummies(train,
                       columns=cat_col)
test = pd.get_dummies(test,
                      columns=cat_col)
train.head()

# %% [markdown]
# <a id='Label Encoding'></a>
# ### Label Encoding

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:52.688493Z","iopub.execute_input":"2024-02-23T07:48:52.688905Z","iopub.status.idle":"2024-02-23T07:48:52.70403Z","shell.execute_reply.started":"2024-02-23T07:48:52.688875Z","shell.execute_reply":"2024-02-23T07:48:52.702078Z"}}
target = 'NObeyesdad'

le = LabelEncoder()
train['NObeyesdad'] = le.fit_transform(train['NObeyesdad'])

# %% [markdown]
# ### Train-Test-Split

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:52.705952Z","iopub.execute_input":"2024-02-23T07:48:52.706394Z","iopub.status.idle":"2024-02-23T07:48:52.72953Z","shell.execute_reply.started":"2024-02-23T07:48:52.706363Z","shell.execute_reply":"2024-02-23T07:48:52.728019Z"}}
X_train, X_val, y_train, y_val = train_test_split(train.drop([target],axis=1),train[target],test_size=0.2,random_state=42)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:52.731317Z","iopub.execute_input":"2024-02-23T07:48:52.731747Z","iopub.status.idle":"2024-02-23T07:48:52.747196Z","shell.execute_reply.started":"2024-02-23T07:48:52.731715Z","shell.execute_reply":"2024-02-23T07:48:52.745778Z"}}
X_train.shape , y_train.shape, X_val.shape, y_val.shape

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:52.749221Z","iopub.execute_input":"2024-02-23T07:48:52.750093Z","iopub.status.idle":"2024-02-23T07:48:52.762931Z","shell.execute_reply.started":"2024-02-23T07:48:52.75005Z","shell.execute_reply":"2024-02-23T07:48:52.761698Z"}}
X_test = test.copy().drop(['id'],axis=1)

# %% [markdown]
# <a id='ModelB'></a>
# # Model building

# %% [markdown]
# <a id='LGBM'></a>
# ## LGBM Model

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:52.764501Z","iopub.execute_input":"2024-02-23T07:48:52.764931Z","iopub.status.idle":"2024-02-23T07:48:54.283005Z","shell.execute_reply.started":"2024-02-23T07:48:52.764899Z","shell.execute_reply":"2024-02-23T07:48:54.28157Z"}}
import optuna
ran_optuna = False

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.284563Z","iopub.execute_input":"2024-02-23T07:48:54.284971Z","iopub.status.idle":"2024-02-23T07:48:54.295998Z","shell.execute_reply.started":"2024-02-23T07:48:54.28494Z","shell.execute_reply":"2024-02-23T07:48:54.294675Z"}}
def optimization_function(trial) :

    lgbParams = {
        'num_class': 7,
        'random_state': 42,
        'metric': 'multi_logloss',
        "boosting_type": "gbdt",
        'objective': 'multiclass',

        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 400, 600),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-1, 10.0),
        'max_depth': trial.suggest_int('max_depth', 6, 20),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.9),
        'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
    }

    lgb_model=lgb.LGBMClassifier(**lgbParams)

#     skf = StratifiedKFold(n_splits=5,shuffle=False, random_state=None)
#     accuracy = cross_val_score(lgb_model,X_train,y_train, cv=skf,scoring='accuracy')
#     print("="*50,'\nValidation Accuracy:', accuracy.mean())

    lgb_model.fit(X_train,y_train)

    acc = accuracy_score(y_val,lgb_model.predict(X_val))

    return acc

# %% [code] {"_kg_hide-output":true,"execution":{"iopub.status.busy":"2024-02-23T07:46:00.05649Z","iopub.execute_input":"2024-02-23T07:46:00.057043Z","iopub.status.idle":"2024-02-23T07:46:00.094625Z","shell.execute_reply.started":"2024-02-23T07:46:00.056998Z","shell.execute_reply":"2024-02-23T07:46:00.09309Z"}}
# %%time

# # Importing Optuna for hyperparameter optimization
# from optuna.samplers import TPESampler
# import optuna

# # Set up the sampler for Optuna optimization
# sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# # Create a study object for Optuna optimization
# study = optuna.create_study(direction="maximize", sampler=sampler)

# # Run the optimization process
# study.optimize(optimization_function, n_trials=200)

# # Get the best parameters after optimization
# best_params = study.best_params

# print('='*50)
# print(best_params)

# ran_optuna = True

# %% [code] {"_kg_hide-output":true,"execution":{"iopub.status.busy":"2024-02-23T07:48:54.297893Z","iopub.execute_input":"2024-02-23T07:48:54.298297Z","iopub.status.idle":"2024-02-23T07:48:54.316126Z","shell.execute_reply.started":"2024-02-23T07:48:54.298262Z","shell.execute_reply":"2024-02-23T07:48:54.314923Z"}}
if ran_optuna :

    print('Number of finished trials:', len(study.trials))

    print('Best trial:', study.best_trial.params)

    optuna.visualization.plot_param_importances(study)

    study.trials_dataframe().sort_values('value',ascending=False)

    optuna.visualization.plot_slice(study)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.318078Z","iopub.execute_input":"2024-02-23T07:48:54.318877Z","iopub.status.idle":"2024-02-23T07:48:54.329087Z","shell.execute_reply.started":"2024-02-23T07:48:54.318839Z","shell.execute_reply":"2024-02-23T07:48:54.327823Z"}}
# 100 trials
# {'objective': 'multiclassova', 'learning_rate': 0.04641200998070569, 'n_estimators': 587, 'reg_alpha': 0.0065043557057678746, 'reg_lambda': 4.460933310544669, 'max_depth': 7, 'colsample_bytree': 0.6833315654013498, 'subsample': 0.8193986843950917, 'min_child_samples': 15}

# %% [markdown]
# <a id='Tuning'></a>
# # Tuning & Retraining

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.330855Z","iopub.execute_input":"2024-02-23T07:48:54.331248Z","iopub.status.idle":"2024-02-23T07:48:54.344304Z","shell.execute_reply.started":"2024-02-23T07:48:54.331215Z","shell.execute_reply":"2024-02-23T07:48:54.342749Z"}}
if ran_optuna :
    lgbParams = study.best_trial.params

else :

#     # 100- traials with PCA seed = None
#     lgbParams = {
#         'objective': 'multiclassova',
#         'learning_rate': 0.04641200998070569,
#         'n_estimators': 587,
#         'reg_alpha': 0.0065043557057678746,
#         'reg_lambda': 4.460933310544669,
#         'max_depth': 7, 'colsample_bytree': 0.6833315654013498,
#         'subsample': 0.8193986843950917,
#         'min_child_samples': 15
#     }


    # Moaz HyperParams
    lgbParams = {
        "objective": "multiclass",          # Objective function for the model
        "metric": "multi_logloss",          # Evaluation metric
        "verbosity": -1,                    # Verbosity level (-1 for silent)
        "boosting_type": "gbdt",            # Gradient boosting type
        "random_state": 42,       # Random state for reproducibility
        "num_class": 7,                     # Number of classes in the dataset
        'learning_rate': 0.030962211546832760,  # Learning rate for gradient boosting
        'n_estimators': 500,                # Number of boosting iterations
        'lambda_l1': 0.009667446568254372,  # L1 regularization term
        'lambda_l2': 0.04018641437301800,   # L2 regularization term
        'max_depth': 10,                    # Maximum depth of the trees
        'colsample_bytree': 0.40977129346872643,  # Fraction of features to consider for each tree
        'subsample': 0.9535797422450176,    # Fraction of samples to consider for each boosting iteration
        'min_child_samples': 26             # Minimum number of data needed in a leaf
    }

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.346401Z","iopub.execute_input":"2024-02-23T07:48:54.346921Z","iopub.status.idle":"2024-02-23T07:48:54.361075Z","shell.execute_reply.started":"2024-02-23T07:48:54.346877Z","shell.execute_reply":"2024-02-23T07:48:54.359534Z"}}
fixed_params = {
    'boosting_type': 'gbdt',
    'num_class': 7,
    'random_state': 42,
    'metric': 'multi_logloss',
}

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.362691Z","iopub.execute_input":"2024-02-23T07:48:54.363078Z","iopub.status.idle":"2024-02-23T07:48:54.374947Z","shell.execute_reply.started":"2024-02-23T07:48:54.363035Z","shell.execute_reply":"2024-02-23T07:48:54.373477Z"}}
for i in fixed_params.keys() :

    lgbParams[i] = fixed_params[i]

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.377342Z","iopub.execute_input":"2024-02-23T07:48:54.377947Z","iopub.status.idle":"2024-02-23T07:48:54.390984Z","shell.execute_reply.started":"2024-02-23T07:48:54.377907Z","shell.execute_reply":"2024-02-23T07:48:54.3895Z"}}
lgbParams

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.392899Z","iopub.execute_input":"2024-02-23T07:48:54.393444Z","iopub.status.idle":"2024-02-23T07:48:54.401838Z","shell.execute_reply.started":"2024-02-23T07:48:54.393407Z","shell.execute_reply":"2024-02-23T07:48:54.400868Z"}}
lgb_model_final = lgb.LGBMClassifier(**lgbParams)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:48:54.408875Z","iopub.execute_input":"2024-02-23T07:48:54.409576Z","iopub.status.idle":"2024-02-23T07:49:33.514464Z","shell.execute_reply.started":"2024-02-23T07:48:54.409524Z","shell.execute_reply":"2024-02-23T07:49:33.512988Z"}}
lgb_model_final = lgb_model_final.fit(X_train, y_train,verbose=100)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:49:33.516671Z","iopub.execute_input":"2024-02-23T07:49:33.517092Z","iopub.status.idle":"2024-02-23T07:49:34.100285Z","shell.execute_reply.started":"2024-02-23T07:49:33.517059Z","shell.execute_reply":"2024-02-23T07:49:34.098474Z"}}
y_pred = lgb_model_final.predict(X_val)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:49:34.102205Z","iopub.execute_input":"2024-02-23T07:49:34.102985Z","iopub.status.idle":"2024-02-23T07:49:34.111944Z","shell.execute_reply.started":"2024-02-23T07:49:34.102945Z","shell.execute_reply":"2024-02-23T07:49:34.110966Z"}}
accuracy_score(y_val, y_pred)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:49:34.113352Z","iopub.execute_input":"2024-02-23T07:49:34.114197Z","iopub.status.idle":"2024-02-23T07:49:34.690624Z","shell.execute_reply.started":"2024-02-23T07:49:34.11416Z","shell.execute_reply":"2024-02-23T07:49:34.689386Z"}}
y_pred_proba = lgb_model_final.predict_proba(X_val)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:49:34.692513Z","iopub.execute_input":"2024-02-23T07:49:34.693323Z","iopub.status.idle":"2024-02-23T07:49:34.735121Z","shell.execute_reply.started":"2024-02-23T07:49:34.69328Z","shell.execute_reply":"2024-02-23T07:49:34.733649Z"}}
X_val = pd.concat([X_val.reset_index(),pd.DataFrame({'actual':le.inverse_transform(y_val),'guess':le.inverse_transform(y_pred)})],axis=1)
X_val['dummy'] = 1
X_val.pivot_table(index='actual',columns='guess',values='dummy',aggfunc='sum')

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:49:34.737189Z","iopub.execute_input":"2024-02-23T07:49:34.737687Z","iopub.status.idle":"2024-02-23T07:49:35.405053Z","shell.execute_reply.started":"2024-02-23T07:49:34.737645Z","shell.execute_reply":"2024-02-23T07:49:35.403557Z"}}
def plot_lgbm_feature_importance(model, feature_names=None, top_n=10, plot=True):

    # Get feature importance
    feature_importance = model.feature_importances_

    # Get feature names
    if feature_names is None:
        feature_names = model.feature_name()

    # Create DataFrame with feature names and importance scores
    feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

    # Sort by importance and select top features
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)#.head(top_n)

    # Plot the feature importance
    if plot:
        plt.figure(figsize=(10, 10))
        sns.barplot(x='Importance', y='Feature', data=feature_importance_df, palette='viridis')
        plt.title('LightGBM Feature Importance')
        plt.show()

    return feature_importance_df

feature_importance_df = plot_lgbm_feature_importance(lgb_model_final,feature_names=X_train.columns)

# %% [markdown]
# <a id='Shap'></a>
# # Plot Shap

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T08:25:14.672028Z","iopub.execute_input":"2024-02-23T08:25:14.672502Z","iopub.status.idle":"2024-02-23T08:25:14.681587Z","shell.execute_reply.started":"2024-02-23T08:25:14.672471Z","shell.execute_reply":"2024-02-23T08:25:14.679652Z"}}
import shap
def plot_shap_summary(model, X, plot_type='bar', plot=True):

    # Create a SHAP explainer object
    explainer = shap.TreeExplainer(model)

    # Calculate SHAP values
    shap_values = explainer.shap_values(X)

    return shap_values

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T08:30:30.617759Z","iopub.execute_input":"2024-02-23T08:30:30.618237Z","iopub.status.idle":"2024-02-23T08:33:16.643187Z","shell.execute_reply.started":"2024-02-23T08:30:30.618202Z","shell.execute_reply":"2024-02-23T08:33:16.641521Z"}}
shap_values = plot_shap_summary(lgb_model_final, pd.DataFrame(data=X_train,columns=X_train.columns))

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T08:33:28.573514Z","iopub.execute_input":"2024-02-23T08:33:28.574166Z","iopub.status.idle":"2024-02-23T08:34:08.684544Z","shell.execute_reply.started":"2024-02-23T08:33:28.574115Z","shell.execute_reply":"2024-02-23T08:34:08.683465Z"},"scrolled":true}
for x in range(len(shap_values)) :
    print("="*50)
    print(le.inverse_transform([x]))
    shap.summary_plot(shap_values[x], pd.DataFrame(data=X_train,columns=X_train.columns)) #, plot_type=plot_type, show=False)

# %% [markdown]
# <a id='Threshold'></a>
# # Optimization of Thresholds

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:18:23.956661Z","iopub.execute_input":"2024-02-23T07:18:23.957116Z","iopub.status.idle":"2024-02-23T07:18:23.962907Z","shell.execute_reply.started":"2024-02-23T07:18:23.957079Z","shell.execute_reply":"2024-02-23T07:18:23.961755Z"}}
threshold_optimization = True
run_optuna_threshold = True

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:18:25.787546Z","iopub.execute_input":"2024-02-23T07:18:25.788012Z","iopub.status.idle":"2024-02-23T07:18:25.79655Z","shell.execute_reply.started":"2024-02-23T07:18:25.787974Z","shell.execute_reply":"2024-02-23T07:18:25.79519Z"}}
import optuna

def objective(trial):

    # Define the thresholds for each class
    thresholds = {}
    for i in range(num_classes):
        thresholds[f'threshold_{i}'] = trial.suggest_uniform(f'threshold_{i}', 0.0, 1.0)

    # Apply the thresholds to convert probabilities to predictions
    y_pred = apply_thresholds(y_pred_proba, thresholds)

    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

def apply_thresholds(y_proba, thresholds):
    # Apply the specified thresholds to convert probabilities to predicted labels
    y_pred_labels = np.argmax(y_proba, axis=1)
    for i in range(y_proba.shape[1]):
        y_pred_labels[y_proba[:, i] > thresholds[f'threshold_{i}']] = i

    return y_pred_labels

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:18:48.293727Z","iopub.execute_input":"2024-02-23T07:18:48.294128Z","iopub.status.idle":"2024-02-23T07:19:17.132967Z","shell.execute_reply.started":"2024-02-23T07:18:48.294096Z","shell.execute_reply":"2024-02-23T07:19:17.1318Z"},"scrolled":true,"_kg_hide-output":true}
if run_optuna_threshold :
    num_classes = 7
    y_pred_proba = y_pred_proba  # Example: replace with actual y_pred_proba
    y_val = y_val  # Example: replace with actual y_val

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=500)

    # Get the best thresholds
    best_thresholds = study.best_params
    print("Best Thresholds:", best_thresholds)
    print("Best Value:", study.best_value)
    threshold_optimization = True


# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:19:30.634475Z","iopub.execute_input":"2024-02-23T07:19:30.635686Z","iopub.status.idle":"2024-02-23T07:19:30.644421Z","shell.execute_reply.started":"2024-02-23T07:19:30.635586Z","shell.execute_reply":"2024-02-23T07:19:30.642696Z"}}
if threshold_optimization :
    if run_optuna_threshold == False :
        best_thresholds = {'threshold_0': 0.724201213234911, 'threshold_1': 0.6161299800571379, 'threshold_2': 0.29138887902587174, 'threshold_3': 0.3145837593497076, 'threshold_4': 0.8469398340837189, 'threshold_5': 0.6800824438387787, 'threshold_6': 0.35886959729223455}

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:19:35.088959Z","iopub.execute_input":"2024-02-23T07:19:35.09025Z","iopub.status.idle":"2024-02-23T07:19:35.100597Z","shell.execute_reply.started":"2024-02-23T07:19:35.090182Z","shell.execute_reply":"2024-02-23T07:19:35.098596Z"}}
# Apply the best thresholds to get final predictions
if threshold_optimization :
    best_y_pred = apply_thresholds(y_pred_proba, best_thresholds)
    accuracy = accuracy_score(y_val, best_y_pred)
    print("Best Accuracy:", accuracy)

# %% [markdown]
# <a id='Pred'></a>
# # Make Predictions & Submission Files

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:22:29.260741Z","iopub.execute_input":"2024-02-23T07:22:29.261579Z","iopub.status.idle":"2024-02-23T07:22:31.534938Z","shell.execute_reply.started":"2024-02-23T07:22:29.261532Z","shell.execute_reply":"2024-02-23T07:22:31.534105Z"}}
y_test_label = lgb_model_final.predict(X_test)

display(len(y_test_label))

test['NObeyesdad'] = y_test_label
out = test[['id','NObeyesdad']]
out.NObeyesdad = le.inverse_transform(out.NObeyesdad)
display(out.head(5))

out.to_csv("submission_non_threshold_opt.csv",index=False)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T07:22:26.746792Z","iopub.execute_input":"2024-02-23T07:22:26.747214Z","iopub.status.idle":"2024-02-23T07:22:29.068118Z","shell.execute_reply.started":"2024-02-23T07:22:26.747179Z","shell.execute_reply":"2024-02-23T07:22:29.067026Z"}}
if threshold_optimization :

    y_test_label = lgb_model_final.predict_proba(X_test)

    y_test_label = apply_thresholds(y_test_label, best_thresholds)

    display(len(y_test_label))

    test['NObeyesdad'] = y_test_label
    out = test[['id','NObeyesdad']]
    out.NObeyesdad = le.inverse_transform(out.NObeyesdad)
    display(out.head(5))

    out.to_csv("submission_threshold_opt.csv",index=False)

# %% [markdown]
# # Missclassification Investigation

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T09:47:26.497681Z","iopub.execute_input":"2024-02-23T09:47:26.498164Z","iopub.status.idle":"2024-02-23T09:47:26.511937Z","shell.execute_reply.started":"2024-02-23T09:47:26.498133Z","shell.execute_reply":"2024-02-23T09:47:26.510593Z"}}
feature_importance_df.head(5).Feature

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:12:30.19479Z","iopub.execute_input":"2024-02-23T10:12:30.195386Z","iopub.status.idle":"2024-02-23T10:12:30.204185Z","shell.execute_reply.started":"2024-02-23T10:12:30.195345Z","shell.execute_reply":"2024-02-23T10:12:30.202744Z"}}
temp = train.copy()

# %% [code]
temp.shape[0]

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:12:30.319132Z","iopub.execute_input":"2024-02-23T10:12:30.319755Z"}}
shap_values = plot_shap_summary(lgb_model_final, pd.DataFrame(data=temp.drop(target,axis=1),columns=temp.columns.drop(target)))

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:08:09.687171Z","iopub.execute_input":"2024-02-23T10:08:09.687613Z","iopub.status.idle":"2024-02-23T10:08:09.825481Z","shell.execute_reply.started":"2024-02-23T10:08:09.687564Z","shell.execute_reply":"2024-02-23T10:08:09.824181Z"}}
shap_magnitude = np.linalg.norm(shap_values,axis=2)
threshold = 6.5
outlier_mask = np.max(shap_magnitude, axis=0) > threshold

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T09:47:29.268434Z","iopub.execute_input":"2024-02-23T09:47:29.268902Z","iopub.status.idle":"2024-02-23T09:47:29.277306Z","shell.execute_reply.started":"2024-02-23T09:47:29.268872Z","shell.execute_reply":"2024-02-23T09:47:29.275681Z"}}
# Use numpy's unique function to get unique values and their counts
unique_values, counts = np.unique(outlier_mask, return_counts=True)

# Combine unique values and counts into a dictionary
value_counts = dict(zip(unique_values, counts))

# Print the value counts
print(value_counts)

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T09:47:33.115731Z","iopub.execute_input":"2024-02-23T09:47:33.116604Z","iopub.status.idle":"2024-02-23T09:47:35.966836Z","shell.execute_reply.started":"2024-02-23T09:47:33.116551Z","shell.execute_reply":"2024-02-23T09:47:35.965681Z"}}
y_test_label = lgb_model_final.predict(temp.drop(target,axis=1))
temp['NObeyesdad_pred'] = y_test_label

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:08:12.353346Z","iopub.execute_input":"2024-02-23T10:08:12.353891Z","iopub.status.idle":"2024-02-23T10:08:12.462267Z","shell.execute_reply.started":"2024-02-23T10:08:12.353853Z","shell.execute_reply":"2024-02-23T10:08:12.460495Z"}}
temp['outlier_mask'] = outlier_mask

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T09:48:25.014479Z","iopub.execute_input":"2024-02-23T09:48:25.014969Z","iopub.status.idle":"2024-02-23T09:48:25.022872Z","shell.execute_reply.started":"2024-02-23T09:48:25.014935Z","shell.execute_reply":"2024-02-23T09:48:25.021446Z"}}
temp['miss_class'] = temp['NObeyesdad_pred']!=temp['NObeyesdad']

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:05:40.969797Z","iopub.execute_input":"2024-02-23T10:05:40.970374Z","iopub.status.idle":"2024-02-23T10:05:40.979352Z","shell.execute_reply.started":"2024-02-23T10:05:40.970334Z","shell.execute_reply":"2024-02-23T10:05:40.977864Z"}}
temp2 = temp[temp['miss_class']]

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:08:01.599199Z","iopub.execute_input":"2024-02-23T10:08:01.599686Z","iopub.status.idle":"2024-02-23T10:08:01.610445Z","shell.execute_reply.started":"2024-02-23T10:08:01.599651Z","shell.execute_reply":"2024-02-23T10:08:01.60913Z"}}
temp['outlier_mask']

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:07:13.61055Z","iopub.execute_input":"2024-02-23T10:07:13.611023Z","iopub.status.idle":"2024-02-23T10:07:13.619979Z","shell.execute_reply.started":"2024-02-23T10:07:13.61099Z","shell.execute_reply":"2024-02-23T10:07:13.618416Z"}}
temp3 = temp[temp['outlier_mask']]

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:05:49.686025Z","iopub.execute_input":"2024-02-23T10:05:49.686453Z","iopub.status.idle":"2024-02-23T10:06:07.262328Z","shell.execute_reply.started":"2024-02-23T10:05:49.686422Z","shell.execute_reply":"2024-02-23T10:06:07.260697Z"}}
# First joint plot
plt.figure(figsize=(8, 8))

# Plot the main scatter plot
sns.kdeplot(x=temp.Height, y=temp.Weight, hue=le.inverse_transform(temp[target]), palette='Set2')

# Add the second set of points as red circles
sns.scatterplot(x='Height', y='Weight', data=temp2, color='red', edgecolor='red', marker='o', linewidth=1, alpha=0.7, facecolors='none')

plt.show()

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:06:07.265096Z","iopub.execute_input":"2024-02-23T10:06:07.266333Z","iopub.status.idle":"2024-02-23T10:06:08.831433Z","shell.execute_reply.started":"2024-02-23T10:06:07.266289Z","shell.execute_reply":"2024-02-23T10:06:08.830392Z"}}
# First joint plot
plt.figure(figsize=(8, 8))

# Plot the main scatter plot
sns.scatterplot(x=temp.Height, y=temp.Weight, hue=le.inverse_transform(temp[target]), palette='Set2')

# Add the second set of points as red circles
sns.scatterplot(x='Height', y='Weight', data=temp2, color='red', edgecolor='red', marker='o', linewidth=1, alpha=0.7, facecolors='none')

plt.show()

# %% [code] {"execution":{"iopub.status.busy":"2024-02-23T10:07:27.514566Z","iopub.execute_input":"2024-02-23T10:07:27.515075Z","iopub.status.idle":"2024-02-23T10:07:29.741944Z","shell.execute_reply.started":"2024-02-23T10:07:27.515043Z","shell.execute_reply":"2024-02-23T10:07:29.740372Z"}}
# First joint plot
plt.figure(figsize=(8, 8))

# Plot the main scatter plot
sns.scatterplot(x=temp.Height, y=temp.Weight, hue=le.inverse_transform(temp[target]), palette='Set2')

# Add the second set of points as red circles
sns.scatterplot(x='Height', y='Weight', data=temp3, color='red', edgecolor='red', marker='o', linewidth=1, alpha=0.7, facecolors='none')

plt.show()